# Variables in Ansible
---

Variables in Ansible can be, and are, used in many different ways. This section aims to cover most of those ways and to offer insight into when and why they are used. But, fear not! Variables in Ansible are easy to use and understand, as we will come to see. 

- You can define, reference and change your own variables
- You can use any of the hundreds of variables that Ansible provides
- Variables fall under different tiers of precedence, covered further at the end of this section
- In most cases, the words 'fact' and 'variable ' are interchangeable in Ansible

> Note:
> - For the information below, refer to the picture under the Modules section

#### Play Meta-Variables
``` yaml
connection: local
```
Specifies what connection type to use when accessing each individual host. eg. SSH, WinRM. `local` means to run on the current machine and not try and SSH into it

``` yaml
gather_facts: false
```
Stops Ansible from gathering the built-in variables. Should be used iif you do not use any of the built in facts. Saves time.

#### Module Variables
- Variables that are specific to each module. Check the Ansible documentation to see what you can and need to set.

#### Task Variables
``` yaml
no_log: true
```
Prevents Ansible from outputting task output to the console

``` yaml
register: variable_name
```
Registers the output of the task to the given variable name

``` yaml
become: true
```
Will run the task with elevated permissions (Linux only)

``` yaml
run_once: true
```
Will run the task on just one host if the current host (group) has multiple hosts

``` yaml
ignore_errors: true
```
Prevents Ansible from crashing when a task fails

``` yaml
delegate_to: 127.0.0.1
```
Delegates the task to the specified host. This is useful when working with tasks that are required to run on a specific OS

## Demos
---

### User-defined Variables
Let's take a look at a simple example of working with variables in Ansible. In the playbook below, we initialize two variables and output them, before changing one's value and outputting both again.

In [ ]:
wpb "ansible_variable_test1.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local


  vars:
    company_name: "Chevron"
    help_desk_email: "help-desk-usa@chevron.com"  # NOTE BOGUS EMAIL ADDRESS FOR DEMONSTRATION ONLY
    help_desk_phone: "CTN: 8-8765"


  tasks:

  - name: set a multiline message
    set_fact:
      my_message: |
        This computer belongs to {{ company_name }}
        For Assistance email: {{ help_desk_email }}
        or call {{ help_desk_phone }}


  - name: Print the new sentence
    debug:
      msg: "{{ my_message.split('\n') }}"

  - name: Change the value using 'set_fact'
    set_fact:
      help_desk_email: "help-desk-asia@chevron.com" # NOTE BOGUS EMAIL ADDRESS FOR DEMONSTRATION ONLY

  - name: set a multiline message
    set_fact:
      my_message: |
        This computer belongs to {{ company_name }}
        For Assistance email: {{ help_desk_email }}
        or call {{ help_desk_phone }}

  - name: Print the new sentence
    debug:
      msg: "{{ my_message.split('\n') }}"
...
############################################################

ansible-playbook "ansible_variable_test1.yml"

### Built-in Facts

In Ansible there are numerous facts that are available to the user simply because they are using Ansible. These variables can be seen by running the `setup` module below.

In [ ]:
ansible localhost -m setup --connection=local

Ansible's built-in variables can be referenced the same as any other variable. Below you can see a playbook that has many examples of what is possible with Ansible's built-in variables.

In [ ]:
wpb "ansible_variable_test2.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  tasks:
  - name: Print all Ansible built-in vars
    setup:

  - name: Set a user-defined variable with the value of a built-in variable
    set_fact:
      my_hostname: "{{ ansible_hostname }}"

  - name: Debug the variable we set from built in var
    debug:
      msg: "{{ my_hostname }}"

  - name: Gather built-in variables using the 'command' module
    command: "hostname"
    register: hostname_output

  - name: Output the gathered 'hostname' command
    debug:
      msg: "{{ hostname_output.stdout }}"
...
############################################################

ansible-playbook "ansible_variable_test2.yml"

### Using built-in facts

#### Why create your own fact when they are built in ?

Ansible's built-in variables are important when it comes to making your code easier to read and write. 

Below is a playbook with two tasks that produce the same result but are vastly different in complexity, one using the `ip addr` command and the other using a built-in Ansible variable.



In [ ]:
wpb "ansible_variable_test3.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  tasks:

    - name: Print the built-in IP variable
      debug:
        msg: "{{ ansible_default_ipv4.address }}"

    - name: Register grep string
      set_fact:
          grp: |
            ip -4 addr show eth0 | grep -oP '(?<=inet\s)\d+(\.\d+){3}'
    - name: Register the IP from 'ip'
      shell: "{{ grp }}"
      register: command_output

    - name: Print the registered IP
      debug:
        msg: "{{ command_output.stdout }}"


...
############################################################

ansible-playbook "ansible_variable_test3.yml"

### An Example

Use the hostname and the IP address of the machine to add to a hosts file.

In [ ]:
wpb "ansible_variable_test4.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  vars:
    tmp_hosts_file: "/home/cvx_admin_user/hosts"
    user_name: "cvx_admin_user"
    group_name: "cvx_admin_user"


  tasks:

    - name: Add my host name and IP address to a file.
      copy:
        dest: "{{ tmp_hosts_file }}"
        content: |
          ## My Hosts file
          {{ ansible_default_ipv4.address }} {{ ansible_facts.fqdn }}
        owner: "{{ user_name }}"
        group: "{{ group_name }}"
        mode: 0644

    - name: show the contents of the file {{ tmp_hosts_file }}
      command: "cat {{ tmp_hosts_file }}"
      register: file_contents_cat

    - name:
      debug:
        msg: "{{ file_contents_cat.stdout_lines }}"

...
############################################################

ansible-playbook "ansible_variable_test4.yml"

/bin/ls -l /home/cvx_admin_user/hosts
cat /home/cvx_admin_user/hosts

### Play and Task variables

Now let's take a look at different Play and Module variable and when they might prove useful.

**1. Run the playbook below**

> Note:
> - `gather_facts: false` is commented
> - the tasks run

**2. Run the playbook below with `gather_facts: false` uncommented**

> Note:
> - the play fails as the facts are not available


In [ ]:
wpb "ansible_variable_test5.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local
  # gather_facts: false

  tasks:
  - name: debug and demonstrate
    debug:
      msg: "{{ ansible_facts.fqdn }}"
...
############################################################

ansible-playbook "ansible_variable_test5.yml"

## Further Explanation
---

**Variable Precedence:**

1. extra vars (always win precedence)
3. role (and include_role) params
4. set_facts / registered vars
6. task vars (only for the task)
8. role vars (defined in role/vars/main.yml)
11. play vars
14. inventory host_vars/*
17. inventory group_vars/*
19. inventory group_vars/all
21. role defaults

**See Also:**
- [Playbook Variables](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html)
- [Special Variables](https://docs.ansible.com/ansible/latest/reference_appendices/special_variables.html)
- [Variable Precedence](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html#variable-precedence-where-should-i-put-a-variable)